<a href="https://colab.research.google.com/github/tanupendeti/repo/blob/main/R9_DTreesVControl_DS110_F25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Tree Decisions

One of the nice things about decision trees is that it's possible to tell, for each data point, exactly the decisions that led to its classification.

Below is a DecisionTree class that has the structure of a decision tree, although we here omit methods that would let us train it.  Code a function print_reasoning() that, given a datapoint represented by a dictionary and a decision tree, prints "[featurename]: [value]" for each node the datapoint encounters as it travels down the tree, followed by the classification when it reaches a leaf node.  Thus the output for the "octopus" dictionary given below should be:

<code>
8 legs: True
Underwater: True
octopus
</code>

We assume all the features here are boolean for convenience, so the object is represented by a dictionary with keys of featurenames and values of True or False.

It is probably easiest to write your function recursively.  Your base case is reaching a leaf and printing the classification.  Your recursive case should print a little of the output and then make a recursive call on the correct branch of the tree.

In [2]:
class DecisionTree:
    def __init__(self, featurename, classification):
        self.featurename = featurename # Is None for leaf
        self.classification = classification # Is None for interior node
        self.yes = None # Link to "yes" branch of tree
        self.no = None # Link to "no" branch of tree

In [3]:
# This sort of tree creation would be done automatically in a
# decision tree learning algorithm
octopus_tree = DecisionTree('8 legs', None)
octopus_tree.yes = DecisionTree('Underwater', None)
octopus_tree.yes.yes = DecisionTree(None, 'octopus')
octopus_tree.yes.no = DecisionTree(None, 'spider')
octopus_tree.no = DecisionTree(None, 'something else')

In [4]:
# TODO print_reasoning()
def print_reasoning(tree, datapoint):
    if tree.classification is not None:
        print(tree.classification)
    else:
        feature_value = datapoint.get(tree.featurename)
        print(f"{tree.featurename}: {feature_value}")
        if feature_value is True:
            print_reasoning(tree.yes, datapoint)
        else:
            print_reasoning(tree.no, datapoint)

In [5]:
octopus = {'8 legs': True, 'Underwater': True}
spider = {'8 legs': True, 'Underwater': False}
print_reasoning(octopus_tree, octopus)
print_reasoning(octopus_tree, spider)

8 legs: True
Underwater: True
octopus
8 legs: True
Underwater: False
spider


# Gini Impurity

In lecture, we learned how entropy was a used as a measure of uncertainty/impurity in a dataset. The formula for entropy is $- \sum_i p_i \log_2 p_i$ where $p_i$ is the proportion of class $i$ in the dataset.

Another measure of impurity is Gini impurity which is defined as

$$
G = 1 - \sum_i p_{i}^{2},
$$

where $p_i$ is again the proportion of class $i$ in the dataset. For binary classification with a 0-class and a 1-class, this formula reduces to

$$
G = 2p(1-p),
$$

where $p$ is the proportion of the 1-class.

Observe that in the binary classification case

- $G=0$ indicates that all elements are of the same class (p=1 or p=0, 100% are of class 1 or 100% are of class 0)
- $G=0.5$ is the maximum impurity, an equal number of 0s and 1s (p=0.5, 50% are of class 1)

Write a function called `gini_impurity(L)` which takes in a list of labels $L$ of 0's and 1's and returns the calculated gini impurity.

In [10]:
#TODO
def gini_impurity(L):
    if not L:
        return None

    n = len(L)
    count_ones = sum(L)
    count_zeros = n - count_ones
    p_one = count_ones / n
    p_zero = count_zeros / n
    gini = 1 - (p_one**2 + p_zero**2)
    return gini

print(gini_impurity([1, 0, 1, 0])) #expect 0.5
print(gini_impurity([])) #expect None
print(gini_impurity([1, 1, 1, 1])) #expect 0
print(gini_impurity([1, 1, 0, 1])) #expect 0.375

0.5
None
0.0
0.375


# Bagging

Bagging, or bootstrap aggregation, is a method of sampling with replacement to generate training samples for an ensemble method like Decision Forests. You are going to implement a function `bagger(L, n)`, which takes as input a list L of data items and a number n of samples to generate. This function will then return a list of lists containing the n samples with replacement of L. The length of each sample should be equal to the length of L.

In the `random` python module, there is a function called `choices(data, k)`, which will sample the data with a sample size of `k`. Use this function in `bagger`. If the input list is empty, return an empty list.

In [14]:
import random
random.seed(42)
#TODO
def bagger(L, n):
  if not L:
    return []
  sample_size = len(L)
  l = []
  for i in range(n):
    l.append(random.choices(L, k=sample_size))
  return l

print(bagger(['a', 'b', 'c', 'd', 'e'], 3))
#expect [['d', 'a', 'b', 'b', 'd'], ['d', 'e', 'a', 'c', 'a'], ['b', 'c', 'a', 'a', 'd']]
print(bagger([], 3)) #expect []
print(bagger(['a', 'b', 'c'], 2)) #expect [['b', 'a', 'b'], ['c', 'a', 'c']]

[['d', 'a', 'b', 'b', 'd'], ['d', 'e', 'a', 'c', 'a'], ['b', 'c', 'a', 'a', 'd']]
[]
[['b', 'a', 'b'], ['c', 'a', 'c']]


# Tool focus:  Version Control

Have you ever worked on a project where you needed to pass around a file to modify?  You probably were a little discontent with the process.  Near the end, it may have been hard to determine whether FinalFinal.docx was really your most recent copy, or if there might be a FinalFinalFinal.docx that was more recent.  If you were collaborating, it may have been annoying to try to keep people from working on the same stuff simultaneously.  And if you wanted to go back to an earlier version, you could only hope you saved a separate version around the right time.

*Version control*, and the git/Github software and site in particular, is the programmer's answer to all the aforementioned problems.

* A central server keeps track of a series of versions of the code.  It's always clear what the most recent version is, and it's relatively easy to go back to an earlier version.  If code becomes buggy, this helps determine when the bug was introduced.

* When people get copies to work on locally, on their own machines, they create their own individual histories locally.  People are free to ignore their collaborators' work until they merge their changes back into the central repository.

* When it's time to merge local changes into the central repository, the code is scanned for differences, and any discrepancies resulting from others' changes are highlighted and resolved.

If you become familiar with version control now, it may especially help with the miniproject, since it will allow you to revert to earlier versions without trying to maintain different versions of the file yourself.

* If you do not already have a github account, go create one now at:  https://github.com/
* Once you've created an account, create a new repository.  If you don't see the big green button immediately, you can click on yourself, then on "Repositories," then "New" in the upper right.
* Name your repository something arbitrary, like "repo_test."  Also make it private (this is what you'd want to do for homework and other sensitive stuff).  Make a README file (because it also sets up a "branch" that you can commit to).  The other options don't really matter right now.
* Assuming you are doing this recitation in Colab, go to File->Save a copy in GitHub.  Choose your notebook and save it to GitHub!  (You might also change the commit message - this is a good habit to get into so that you can find particular versions later.)
* Check the repo on the GitHub website (under your username->Repositories), and verify that your code is now on the web (privately, assuming you chose "private").

* Now let's try going back to an earlier version.  Fill the following code box with junk, and File->Save a copy in GitHub.

* You can now see both versions of the notebook in the GitHub repository history.  (Repositories->your_repo_name->your_filename->History, with the clock icon.)
* To open the most recent version, go to "File->Open Notebook" in Colab and choose the GitHub tab at the top.  Choose your repo and open it.
* You can also access the versions in your history from Colab.  Go to File->Revision History.  (Notice that the right-hand side displays differences between the files.)  The dots next to each item in the history have the option of Open in Colab.  Try restoring an old version now.

* Last, try collaborating with someone on this file.  Within your repo on GitHub, go to Settings->Collaborators->Add people.  Add someone in your section who is willing to share their username with you.  "Sign" each others' work in the last text box, and show this along with your completed code to the TA to finish this section.

**Your collaborator's signature here**